# input

In [1]:
search_term=["경제 호재"]
output_file="prediction.csv"
input_path="[이 파일이 있는 경로]"

# import

In [2]:
import requests
import json
import pandas as pd
import csv
import glob
import os
import sys
from IPython.display import HTML

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# 기사 텍스트 추출

In [3]:
# (Bing news Search)애플리케이션 생성 및 초기화
subscription_key = "[Bing News Search key]"
search_url = "https://api.bing.microsoft.com/v7.0/news/search?q=sailing&count=40"

# (Text_Analisis)클라이언트 인증
key = "[Text Analitics key]"
endpoint = "https://textanalysis-2.cognitiveservices.azure.com/"

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

In [4]:
def entity_recognition_example(client):

    try:
        documents = [""]
        documents[0]+=rows1
        result1 = client.recognize_entities(documents = documents, language="ko")[0]
        
        documents = [""]
        documents[0]+=rows2
        result2 = client.recognize_entities(documents = documents, language="ko")[0]

        for entity in result1.entities:
            EntityList.append(entity.text)
            
        for entity in result2.entities:
            EntityList.append(entity.text)
            
    except Exception as err:
        print("Encountered exception. {}".format(err))

In [5]:
headers = {"Ocp-Apim-Subscription-Key" : subscription_key}
params  = {"q": search_term[0], "textDecorations": True, "textFormat": "HTML"}

# 요청보내고 응답받기
response = requests.get(search_url, headers=headers, params=params)
response.raise_for_status()
search_results = json.dumps(response.json(), indent=4)
search_results = json.loads(search_results)

descriptions = [article["description"] for article in search_results["value"]]

# HTML 출력
rows = "\n".join(["<tr><td>{0}</td></tr>".format(desc)
                  for desc in descriptions])
display(HTML("<table>"+rows+"</table>"))

# row 두개로 나눠서 저장
rows1 = []
rows2 = []
DeLen = int(len(descriptions)/2)
rows1 = "\n".join(["{0}\n".format(descriptions[i]) for i in range(DeLen)])
rows2 = "\n".join(["{0}\n".format(descriptions[i+DeLen]) for i in range(DeLen)])

# 명명된 엔터티 인식
EntityList=[]
entity_recognition_example(client)

현대엘리베이터가 전반적인 수익성을 개선할 것으로 예상된다. 지난해 하반기부터 주택건설 및 승강기 신규 설치량 증가세로 전환된 데 이어 정부 주택공급 확대 정책의 수혜를 입을 것이라는 분석이다. 31일 NH투자증권은 이 같은 배경에 현대엘리베이터의 투자의견 '매수'를 유지하고 목표가를 6만8000원으로 21.4% 상향했다. 전거래일 종가는 5만3500원이었다
3기 신도시 중 최대 규모인 경기 남양주 왕숙신도시에 지하철 9호선이 연장된다. 서울 강일동과 하남을 거치는 노선으로 2028년 개통이 목표다. 이에 발 빠른 투자자들은 남양주의 수년 뒤 미래가치를 내다보고 있다. 부동산 전문가들은 “9호선이 연장되면 왕숙신도시는 물론 인근 남양주 다산 지금지구까지 지역 부동산 가치가 확 뛸 것”이라고 입을 모은다. 9호선
서울 용산에서 진행 중인 굵직한 개발 프로젝트들이 잇따라 속도를 내고 있다. 30일 서울시에 따르면 시는 내달 중 용산 유엔사 부지를 건축위원회 심의에 상정할 방침이다. 시 관계자는 “심의 상정을 위한 준비가 거의 다 끝났다”며 “오는 6월 중 심의를 요청할 계획”이라고 말했다. 용산구 이태원동 유엔사 부지는 용산공원과 이태원을 연결하는 완충지이자 용산공원
기타리스트 겸 싱어송라이터 이상순이 첫 솔로 EP ‘이상순(Leesangsoon)’를 발표한다. 이상순은 오는 6월 4일 각종 온라인 음원 사이트를 통해 본인의 이름을 타이틀로 한 EP ‘이상순’을 발매할 예정이다. 이번 앨범은 디지털 싱글을 제외하면 이상순의 솔로 명의로 발표하는 첫 앨범이다. 2001년 김동률과 함께한 베란다 프로젝트의 ‘데이 오프(Da
다음주(5월 31일~6월 4일) 국내 증시는 호재와 악재 간 힘겨루기 과정에서 일시적인 변동성이 동반되겠다.30일 한국거래소에 따르면 지난주(5월 24~28일) 국내 증시는 하락세로 시작해 상승 마감에 성공했다. 종가 기준 코스피지수는 24일 전 거래일 대비 12.12포인트(-0.38%) 떨어진 3144.30에 한 주를 시작해 등락을 거듭하다 28일 23.
일부 알트코인이 특별한 호재 없이 폭등하는 현상이 벌어지고 있다. 일부 코인은 하루 만에 100~300% 크게 올랐다. 21일 가상화폐 거래소 업비트에 따르면 오후 3시 35분 현재 센티넬프로토콜은 전날보다 140% 상승한 300원에 거래되고 있다. 센티넬프로토콜은 전날 저점(100원)과 비교하면 이날 한때 386원으로 286% 치솟았다. 엘비알와이크레딧도
"[서울경제TV=김수빈기자]대신증권은 20일 LS전선아시아(229640)에 대해 “구리 가격 상승을 통한 반사이익이 발생하고 해저케이블의 신성장 요인이 부각되는 등의 호재가 많다”며 투자의견은 매수로, 목표주가는 1.."
"이란 대통령 ""비트코인 불법 채굴로 국가 전력난"" ""향후 4개월, 9월22일까지 코인 채굴 전면 금지"" 블랙록 ""암호화폐 투자 가능한 자산인지 검토 중"" 가상(암호)화폐 시장이 악재와 호재의 동시 등장에 명확한 방향성을 찾지 못하고 있다. 중국, 미국, 이란 등 각국 정부가 암호화폐에 대한 규제 강도를 높이는 사이 대규모 자산운용사들은 암호화폐에 대한 투자"
"테슬라에 대한 세가지 우려를 자세히 살펴보자. 먼저 중국시장에 대한 우려는 2021년 초부터 제기되었다. 중국정부는 정보보안을 이유로 테슬라 차량의 군사지역 출입을 금지 시켰다. 테슬라는 상하이에 데이터센터를 설립하고, 중국 밖으로 데이터가 나가지 않을 것 임을 약속했지만, 자율주행 기술은 자국업체를 중심으로 발전할 수 밖에 없음을 보여주는 계기가 되었다."
"권덕철 보건복지부 장관과 정의용 외교부 장관, 문승욱 산업통상자원부 장관(오른쪽부터)이 25일 정부서울청사 별관 외교부 브리핑룸에서 열린 한미 정상회담 결과 정부합동 온라인 브리핑을 마친 뒤 포즈를 취하고 있다. 외교부 제공. 문승욱 산업통상자원부 장관은 25일 ""이번 한미 정상회담에서 논의한 어젠다는 특정국과 관련된 사안은 아니다""라며 ""중국은 우리 수출"
"[현대경제신문 정유라 기자] 대우건설은 다음달 대구광역시 동구 용계동에 ‘용계역 푸르지오 아츠베르’를 분양한다고 18일 밝혔다. 용계동 도시개발사업으로 추진되는 해당 단지는 A1·A2블록 2개 단지로 조성된다. 지하 2층~지상 15층, 총 21개 동, 전용 59㎡~99㎡ 총 1천313가구 규모다. 용계역 푸르지오 아츠베르는 인근 율하 권역 내 신규 분양 단"


# Sector별 데이터 프레임 불러오기

In [6]:
weight_list=[]
weight_acc_list=[]
RateValue=[]
all_data_frames=[]

# 불러오기
all_files = glob.glob(os.path.join(input_path, 'sector_*'))
all_data_frame=[]
for num,input_file in enumerate(all_files):
    data_frame=pd.read_csv(input_file, index_col=None, encoding='CP949')
    
    # 이름 불러오기
    total_name = [value for value in data_frame.loc[:,'phrase']]
        
    # 비교
    count=0
    for entity in EntityList:
        for i,name in enumerate(total_name):
            if entity == name:
                count+=data_frame.loc[i,'weight']
                    
    # total 저장
    total_weight = sum([value for value in data_frame.loc[:,'weight']]) # 누적
    weight_list.append(count) # 
    weight_acc_list.append(total_weight)
    RateValue.append(count/total_weight)
    
    # 데이터프레임화
    data = {'file_name':os.path.basename(input_file),
            'rate_value':count/total_weight}
    all_data_frames.append(pd.DataFrame(data,index=[num+1]))
    
# 저장
data_frames_concat = pd.concat(all_data_frames, axis=0, ignore_index=True)
data_frames_concat.to_csv(output_file, index=False)

# 출력
print(all_data_frames)

[                  file_name  rate_value
1  sector_semiconductor.csv    0.807263,             file_name  rate_value
2  sector_battery.csv    1.122807,         file_name  rate_value
3  sector_car.csv    1.019417,          file_name  rate_value
4  sector_Baio.csv    0.478405,          file_name  rate_value
5  sector_iron.csv    0.423237,          file_name  rate_value
6  sector_bank.csv    0.574074]
